In [2]:
OVER_TIME_DIR = 'over_time-1883'
KEYWORD = 'jobs'
DMA_Code = 'US-HI-744' # Honolulu HI
DMA_Str = 'Honolulu HI'

In [1]:
import pandas as pd
import glob

In [3]:
sorted(glob.glob("{}/{}/0-{}-*.csv".format(OVER_TIME_DIR, KEYWORD, DMA_Code)))

[]

# Concatenate time-series

In [4]:
dfs = []
for afile in sorted(glob.glob("{}/{}/0-{}-*.csv".format(OVER_TIME_DIR, KEYWORD, DMA_Code))):
    dfs.append(pd.read_csv(afile))

In [5]:
pivot = dfs[0]
for i in range(1, len(dfs)):
    new_df = dfs[i]
    pivot_multiplier = 100/pivot.iloc[-1][KEYWORD]
    new_multiplier = 100/new_df.iloc[0][KEYWORD]
    
    pivot.iloc[:,1] = pivot.iloc[:,1] * pivot_multiplier
    new_df.iloc[:,1] = new_df.iloc[:,1] * new_multiplier
    
    pivot = pd.concat([pivot.iloc[:-1,:], new_df])
pivot.to_csv("reference_{}_{}_{}.csv".format(KEYWORD, DMA_Code, OVER_TIME_DIR), sep=",", index=False)

In [6]:
timeline = pivot

# Rescale cross-sectional indices

In [7]:
import os
CONVERT_DIR = "converted-{}".format(DMA_Code)
try:
    os.makedirs("{}/{}".format(CONVERT_DIR, KEYWORD))
except:    
    pass

In [8]:
import os

for row_index, row in timeline.iterrows():
    week_start = row['date']
    week_value = row[KEYWORD]
    
    snapshot_file = glob.glob("by_region/{}/0-DMA-{}-*.csv".format(KEYWORD, week_start.replace("-", "")))[0]
    df = pd.read_csv(snapshot_file)
    convert_multiplier = float(row[KEYWORD]/df[df['geoName'].str.contains('{}'.format(DMA_Str))][KEYWORD])
#     print (convert_multiplier)
    
    df.iloc[:,1] = df.iloc[:,1] * convert_multiplier
    df.to_csv("{}/{}/{}".format(CONVERT_DIR, KEYWORD, os.path.basename(snapshot_file)))
#     break